In [1]:
#Import files module
import pandas as pd
import csv
import re                     ##Regular expressions
import glob
import os
import logging
import numpy as np
import datetime
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from math import sqrt
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

C:\Users\Lenovo\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Lenovo\Anaconda2\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implement

In [2]:
#logging mechanism for debugging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
#Mapping months to numeric values
months = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'June','07':'July','08':'Aug','09':'Sept','10':'Oct','11':'Nov','12':'Dec'}

In [25]:
import pandas as pd

#Importing files from specific folders and list of holidays
bus_no='6020'                                                 #Route number - Change this value accordingly 
path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Up'     #Up Direction files folder Path - Change the path accordingly    
down_path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Down'    #Down Direction files folder path - Change the path accordingly
allfiles = glob.glob(path + "/*.csv")
down_allfiles = glob.glob(down_path + "/*.csv")
Holiday_colnames = ['Holiday', 'Date', 'Day', 'Month']
Holiday_data = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays.csv', names=Holiday_colnames)     #Holiday list for 2018
Holiday_2017 = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays 2017.csv', names=Holiday_colnames)      #Holiday list for 2017
feature_names =  ['Time Slot','Trips','Passengers','Day','isHoliday']

# Up direction processing

In [26]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

#Dicing only requird data from list of holidays
holiday_day = Holiday_data['Day']
holiday_month = Holiday_data['Month']
holiday_day_2017 = Holiday_2017['Day']
holiday_month_2017 = Holiday_2017['Month']

In [27]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,23):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][29:]
    date = os.path.splitext(base)[0][26:28]

    month = os.path.splitext(base)[0][23:25]
    year = os.path.splitext(base)[0][18:22]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [28]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


rmse_total = 0
rmse_total_train = 0
forecasted_trips = []
# load dataset
for i in range(0,20):    
    dataset = df[i].copy()
    dataset.drop('Time Slot',axis=1,inplace=True)
    values = dataset.values
    # ensure all data is float
    values = values.astype('float32')
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    # frame as supervised learning
    reframed = series_to_supervised(scaled, 1, 1)
    # drop columns we don't want to predict
    reframed.drop(reframed.columns[[5,6,7]], axis=1, inplace=True)
    #print(reframed.head())             #To debug
    
    #Divide Data into Training and test set
    values = reframed.values
    #n_train_steps = 30                  #Training set size
    train_set = values[:, :]
    test_set = values[:, :]
    # split into input and outputs
    train_X, train_y = train_set[:-1,:-1], train_set[:-1,-1]
    test_X, test_y = test_set[-1:,:-1], test_set[-1:,-1]
    
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    
    # design network
    model = Sequential()
    model.add(LSTM(5, input_shape=(train_X.shape[1], train_X.shape[2])))    #Adding LSTM units into the RNN
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')                             #mae-Mean Absolute error, Adam optimization rule
    # fit network
    history = model.fit(train_X, train_y, epochs=20, batch_size=3, validation_data=(test_X, test_y), verbose=1, shuffle=False)
    
    #obtaining predictions
    test_prediction = model.predict(test_X)
    train_prediction = model.predict(train_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    train_X = train_X.reshape((train_X.shape[0], train_X.shape[2]))

    # invert scaling for forecast
    print test_prediction
    actual_prediction = concatenate((test_prediction, test_X[:, 1:]), axis=1)
    actual_prediction = scaler.inverse_transform(actual_prediction)
    actual_prediction = actual_prediction[:,0]
    for x in actual_prediction:
        print round(x)
        if round(x)<=0:
            forecasted_trips.append(0)
        else:
            forecasted_trips.append(round(x))
    actual_train_prediction = concatenate((train_prediction, train_X[:, 1:]), axis=1)
    actual_train_prediction = scaler.inverse_transform(actual_train_prediction)
    actual_train_prediction = actual_train_prediction[:,0]
    
    #invert scaling for actual ie. already available predictions(teacher signal)
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]
    print inv_y
    #print df[i]['Trips']
    
     #invert scaling for actual ie. already available predictions(teacher signal) for training set
    train_y = train_y.reshape((len(train_y),1))
    inv_y_train = concatenate((train_y, train_X[:, 1:]), axis=1)
    inv_y_train = scaler.inverse_transform(inv_y_train)
    inv_y_train = inv_y_train[:,0]
    
    # calculate RMSE
    rmse = sqrt(mean_squared_error(inv_y,actual_prediction))
    rmse_total += float(rmse)
    print "For df ",i,":", ('Test RMSE: %.3f' % rmse)
    
    rmse_train = sqrt(mean_squared_error(inv_y_train, actual_train_prediction))
    rmse_total_train += float(rmse_train)
    #print "For df ",i,":", ('Train RMSE: %.3f' % rmse_train)
print rmse_total/20.0 #, rmse_total_train/20.0

Train on 135 samples, validate on 1 samples
Epoch 1/20
135/135 [==============================] - 8s 62ms/step - loss: 0.0860 - val_loss: 0.0949
Epoch 2/20
135/135 [==============================] - 7s 54ms/step - loss: 0.0797 - val_loss: 0.0772
Epoch 3/20
135/135 [==============================] - 7s 50ms/step - loss: 0.0764 - val_loss: 0.0603
Epoch 4/20
135/135 [==============================] - 8s 56ms/step - loss: 0.0738 - val_loss: 0.04620s - loss: 0.075
Epoch 5/20
135/135 [==============================] - 5s 35ms/step - loss: 0.0723 - val_loss: 0.0340A: 1
Epoch 6/20
135/135 [==============================] - 5s 35ms/step - loss: 0.0716 - val_loss: 0.0238
Epoch 7/20
135/135 [==============================] - 5s 35ms/step - loss: 0.0710 - val_loss: 0.0137
Epoch 8/20
135/135 [==============================] - 5s 36ms/step - loss: 0.0704 - val_loss: 0.0041
Epoch 9/20
135/135 [==============================] - 5s 36ms/step - loss: 0.0697 - val_loss: 0.0062
Epoch 10/20
135/135 [======

Epoch 18/20
135/135 [==============================] - 5s 33ms/step - loss: 0.1091 - val_loss: 0.1331
Epoch 19/20
135/135 [==============================] - 4s 33ms/step - loss: 0.1088 - val_loss: 0.1365
Epoch 20/20
135/135 [==============================] - 4s 33ms/step - loss: 0.1085 - val_loss: 0.1399
[[ 0.40305117]]
11.0
[ 8.]
For df  3 : Test RMSE: 2.658
Train on 135 samples, validate on 1 samples
Epoch 1/20
135/135 [==============================] - 4s 33ms/step - loss: 0.2476 - val_loss: 0.2845
Epoch 2/20
135/135 [==============================] - 6s 45ms/step - loss: 0.1783 - val_loss: 0.2131
Epoch 3/20
135/135 [==============================] - 7s 55ms/step - loss: 0.1364 - val_loss: 0.1554
Epoch 4/20
135/135 [==============================] - 8s 59ms/step - loss: 0.1155 - val_loss: 0.1254
Epoch 5/20
135/135 [==============================] - 8s 59ms/step - loss: 0.1091 - val_loss: 0.1114
Epoch 6/20
135/135 [==============================] - 7s 55ms/step - loss: 0.1076 - val_l

130/130 [==============================] - 5s 36ms/step - loss: 0.1756 - val_loss: 0.2367
Epoch 11/20
130/130 [==============================] - ETA: 0s - loss: 0.176 - 4s 32ms/step - loss: 0.1752 - val_loss: 0.2337
Epoch 12/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1749 - val_loss: 0.2289
Epoch 13/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1746 - val_loss: 0.2250
Epoch 14/20
130/130 [==============================] - 4s 34ms/step - loss: 0.1742 - val_loss: 0.2217
Epoch 15/20
130/130 [==============================] - 4s 34ms/step - loss: 0.1740 - val_loss: 0.2166
Epoch 16/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1737 - val_loss: 0.2124
Epoch 17/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1733 - val_loss: 0.2087
Epoch 18/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1730 - val_loss: 0.2050
Epoch 19/20
130/130 [==============================] - 4s 33ms/step - 

Epoch 3/20
130/130 [==============================] - 4s 32ms/step - loss: 0.0135 - val_loss: 0.0151
Epoch 4/20
130/130 [==============================] - 4s 33ms/step - loss: 0.0067 - val_loss: 8.3453e-04
Epoch 5/20
130/130 [==============================] - 4s 32ms/step - loss: 0.0011 - val_loss: 0.0022
Epoch 6/20
130/130 [==============================] - 4s 32ms/step - loss: 3.2671e-04 - val_loss: 0.0019
Epoch 7/20
130/130 [==============================] - 4s 33ms/step - loss: 3.5323e-04 - val_loss: 7.3108e-04
Epoch 8/20
130/130 [==============================] - 4s 34ms/step - loss: 3.7192e-04 - val_loss: 0.0019
Epoch 9/20
130/130 [==============================] - 4s 32ms/step - loss: 2.6724e-04 - val_loss: 0.0010
Epoch 10/20
130/130 [==============================] - 4s 33ms/step - loss: 3.6705e-04 - val_loss: 0.0029
Epoch 11/20
130/130 [==============================] - 4s 32ms/step - loss: 2.6710e-04 - val_loss: 0.0022
Epoch 12/20
130/130 [==============================] - 5s

In [31]:
print forecasted_trips

[0, 2.0, 7.0, 11.0, 9.0, 7.0, 7.0, 7.0, 7.0, 8.0, 7.0, 6.0, 8.0, 9.0, 8.0, 7.0, 6.0, 4.0, 2.0, 0]


# Down Direction Processing

In [32]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

#Dicing only requird data from list of holidays
holiday_day = Holiday_data['Day']
holiday_month = Holiday_data['Month']

In [33]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,24):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in down_allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][31:]
    date = os.path.splitext(base)[0][29:31]
    month = os.path.splitext(base)[0][26:28]
    
    #Determining whether holiday or not
    year = os.path.splitext(base)[0][21:25]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [34]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


rmse_total = 0
rmse_total_train = 0
forecasted_down_trips =[]
# load dataset
for i in range(0,20):    
    dataset = df[i].copy()
    dataset.drop('Time Slot',axis=1,inplace=True)
    values = dataset.values
    # ensure all data is float
    values = values.astype('float32')
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    # frame as supervised learning
    reframed = series_to_supervised(scaled, 1, 1)
    # drop columns we don't want to predict
    reframed.drop(reframed.columns[[5,6,7]], axis=1, inplace=True)
    #print(reframed.head())             #To debug
    
    #Divide Data into Training and test set
    values = reframed.values
    #n_train_steps = 30                  #Training set size
    train_set = values[:, :]
    test_set = values[:, :]
    # split into input and outputs
    train_X, train_y = train_set[:-1,:-1], train_set[:-1,-1]
    test_X, test_y = test_set[-1:,:-1], test_set[-1:,-1]
    
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    
    # design network
    model = Sequential()
    model.add(LSTM(5, input_shape=(train_X.shape[1], train_X.shape[2])))    #Adding LSTM units into the RNN
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')                             #mae-Mean Absolute error, Adam optimization rule
    # fit network
    history = model.fit(train_X, train_y, epochs=20, batch_size=3, validation_data=(test_X, test_y), verbose=1, shuffle=False)
    
    #obtaining predictions
    test_prediction = model.predict(test_X)
    train_prediction = model.predict(train_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    train_X = train_X.reshape((train_X.shape[0], train_X.shape[2]))

    # invert scaling for forecast
    print test_prediction
    actual_prediction = concatenate((test_prediction, test_X[:, 1:]), axis=1)
    actual_prediction = scaler.inverse_transform(actual_prediction)
    actual_prediction = actual_prediction[:,0]
    for x in actual_prediction:
        print round(x)
        if round(x)<=0:
            forecasted_down_trips.append(0)
        else:
            forecasted_down_trips.append(round(x))
    actual_train_prediction = concatenate((train_prediction, train_X[:, 1:]), axis=1)
    actual_train_prediction = scaler.inverse_transform(actual_train_prediction)
    actual_train_prediction = actual_train_prediction[:,0]
    
    #invert scaling for actual ie. already available predictions(teacher signal)
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]
    print inv_y
    #print df[i]['Trips']
    
     #invert scaling for actual ie. already available predictions(teacher signal) for training set
    train_y = train_y.reshape((len(train_y),1))
    inv_y_train = concatenate((train_y, train_X[:, 1:]), axis=1)
    inv_y_train = scaler.inverse_transform(inv_y_train)
    inv_y_train = inv_y_train[:,0]
    
    # calculate RMSE
    rmse = sqrt(mean_squared_error(inv_y,actual_prediction))
    rmse_total += float(rmse)
    print "For df ",i,":", ('Test RMSE: %.3f' % rmse)
    
    rmse_train = sqrt(mean_squared_error(inv_y_train, actual_train_prediction))
    rmse_total_train += float(rmse_train)
    #print "For df ",i,":", ('Train RMSE: %.3f' % rmse_train)
print rmse_total/20.0 #, rmse_total_train/20.0

Train on 135 samples, validate on 1 samples
Epoch 1/20
135/135 [==============================] - 5s 36ms/step - loss: 0.0896 - val_loss: 0.0011
Epoch 2/20
135/135 [==============================] - 5s 34ms/step - loss: 0.0860 - val_loss: 7.2368e-04
Epoch 3/20
135/135 [==============================] - 4s 33ms/step - loss: 0.0839 - val_loss: 6.9084e-04
Epoch 4/20
135/135 [==============================] - 5s 36ms/step - loss: 0.0832 - val_loss: 7.0041e-05
Epoch 5/20
135/135 [==============================] - 5s 39ms/step - loss: 0.0832 - val_loss: 4.6113e-04
Epoch 6/20
135/135 [==============================] - 5s 37ms/step - loss: 0.0831 - val_loss: 8.3649e-05
Epoch 7/20
135/135 [==============================] - 4s 33ms/step - loss: 0.0832 - val_loss: 2.0249e-04
Epoch 8/20
135/135 [==============================] - 5s 33ms/step - loss: 0.0830 - val_loss: 5.6397e-04
Epoch 9/20
135/135 [==============================] - 5s 33ms/step - loss: 0.0830 - val_loss: 2.5405e-04
Epoch 10/20
135

Epoch 17/20
135/135 [==============================] - 4s 33ms/step - loss: 0.0913 - val_loss: 0.0383
Epoch 18/20
135/135 [==============================] - 4s 33ms/step - loss: 0.0909 - val_loss: 0.0399
Epoch 19/20
135/135 [==============================] - 4s 33ms/step - loss: 0.0907 - val_loss: 0.0431
Epoch 20/20
135/135 [==============================] - 4s 32ms/step - loss: 0.0907 - val_loss: 0.0449
[[ 0.2721262]]
8.0
[ 6.99999952]
For df  3 : Test RMSE: 0.987
Train on 135 samples, validate on 1 samples
Epoch 1/20
135/135 [==============================] - 4s 33ms/step - loss: 0.2511 - val_loss: 0.1506
Epoch 2/20
135/135 [==============================] - 4s 33ms/step - loss: 0.1621 - val_loss: 0.0691
Epoch 3/20
135/135 [==============================] - 4s 33ms/step - loss: 0.1012 - val_loss: 0.0119
Epoch 4/20
135/135 [==============================] - 4s 33ms/step - loss: 0.0866 - val_loss: 0.0162
Epoch 5/20
135/135 [==============================] - 4s 33ms/step - loss: 0.0842 

130/130 [==============================] - 4s 33ms/step - loss: 0.1606 - val_loss: 0.2332
Epoch 10/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1579 - val_loss: 0.2300
Epoch 11/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1560 - val_loss: 0.2206
Epoch 12/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1551 - val_loss: 0.2152
Epoch 13/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1541 - val_loss: 0.2140
Epoch 14/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1531 - val_loss: 0.2102
Epoch 15/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1522 - val_loss: 0.2053
Epoch 16/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1515 - val_loss: 0.2004
Epoch 17/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1508 - val_loss: 0.1954
Epoch 18/20
130/130 [==============================] - 4s 33ms/step - loss: 0.1500 - val_loss:

Epoch 2/20
130/130 [==============================] - 4s 33ms/step - loss: 0.0871 - val_loss: 0.0184
Epoch 3/20
130/130 [==============================] - 4s 34ms/step - loss: 0.0782 - val_loss: 0.0066
Epoch 4/20
130/130 [==============================] - 4s 33ms/step - loss: 0.0715 - val_loss: 0.0013
Epoch 5/20
130/130 [==============================] - 4s 33ms/step - loss: 0.0709 - val_loss: 3.1030e-04
Epoch 6/20
130/130 [==============================] - 4s 34ms/step - loss: 0.0708 - val_loss: 0.0013
Epoch 7/20
130/130 [==============================] - 4s 33ms/step - loss: 0.0710 - val_loss: 1.3136e-04
Epoch 8/20
130/130 [==============================] - 4s 33ms/step - loss: 0.0705 - val_loss: 0.0012
Epoch 9/20
130/130 [==============================] - 4s 33ms/step - loss: 0.0703 - val_loss: 7.2203e-04
Epoch 10/20
130/130 [==============================] - 4s 33ms/step - loss: 0.0707 - val_loss: 4.0031e-04
Epoch 11/20
130/130 [==============================] - 4s 33ms/step - loss

In [35]:
print forecasted_trips
print forecasted_down_trips
print len(forecasted_trips)
print len(forecasted_down_trips)
#forecasted_trips=[]
up_pass=[]                   # Removing irrelevant internal processing data
#forecasted_down_trips=[]
down_pass=[]                 # Removing irrelevant internal processing data

[0, 2.0, 7.0, 11.0, 9.0, 7.0, 7.0, 7.0, 7.0, 8.0, 7.0, 6.0, 8.0, 9.0, 8.0, 7.0, 6.0, 4.0, 2.0, 0]
[0, 0, 5.0, 8.0, 9.0, 8.0, 8.0, 7.0, 7.0, 5.0, 5.0, 7.0, 7.0, 8.0, 8.0, 8.0, 8.0, 5.0, 4.0, 0]
20
20


In [36]:
print len(timeslot)
print len(forecasted_down_trips)
print len(forecasted_trips)

20
20
20


In [37]:
#Saving Results into CSV files 
forecast = pd.DataFrame(columns=['Time Slot','Up_Trips','Up_pass','Down_Trips','Down_pass'])
save_path=r'D:/Dharmit/Final Year Project/Forecasted Trips/'
for j in xrange(len(timeslot)):
    forecast= forecast.append(pd.Series([timeslot[j],forecasted_trips[j],forecasted_down_trips[j]], index = ['Time Slot','Up_Trips','Down_Trips']), ignore_index=True)
filename ='Forecasted_trips_'+bus_no+'_'+str(now.month).zfill(2) +'_'+str(now.day)+'_'+str(day)
forecast.to_csv(save_path + filename +'.csv')